categorical_L3 のハッシュを取って製品を分け、陰性の製品の代表をいくつか取ってそれらとの差分をまとめ、そのminを取る

In [1]:
import pandas as pd
import numpy as np
import hashlib

In [208]:
categorical = pd.read_hdf("../hdf/train_categorical.hdf")

In [207]:
column = [c for c in categorical.columns if "L3" in c]

In [170]:
categorical = categorical.loc[:, column]

In [209]:
response = pd.read_hdf("../hdf/train_response.hdf")

In [210]:
categorical["L3_md5"] = categorical.apply(
    lambda x: int(hashlib.md5(x.values).hexdigest(), 16),
    axis=1).astype(np.float32)

In [211]:
categorical = pd.concat([categorical, response], axis=1)

In [212]:
x1 = pd.DataFrame(categorical[categorical["Response"] == 1.0]["L3_md5"]
                  .value_counts()).rename(columns={
                      "L3_md5": "md1"
                  })
x0 = pd.DataFrame(categorical[categorical["Response"] == 0.0]["L3_md5"]
                  .value_counts()).rename(columns={
                      "L3_md5": "md0"
                  })
x10 = pd.concat([x1, x0], axis=1)
x10["ratio"] = x10["md1"] / (x10["md0"] + x10["md1"])
x10.sort_values(["md0"], ascending=[0])

,md1,md0,ratio
1.096518e+38,2108.0,475464.0,0.004414
1.282007e+38,1005.0,228181.0,0.004385
1.698639e+38,228.0,47978.0,0.004730
1.993187e+38,126.0,30169.0,0.004159
2.691518e+38,207.0,29448.0,0.006980
1.830904e+37,207.0,29286.0,0.007019
5.688920e+37,218.0,29230.0,0.007403
2.697728e+38,153.0,26821.0,0.005672
2.830633e+38,78.0,11432.0,0.006777
2.321118e+38,73.0,10966.0,0.006613


In [215]:
# 400 以上存在する hash を集めます
a0=x10[["md0"]]
famous_hashes = a0[a0["md0"] >= 5000].index.values

In [218]:
# 400 以上存在する hash を出す index を貯めます
famous_ids = [categorical[categorical["L3_md5"] == hash_num].index.values[0]
              for hash_num in famous_hashes]

In [219]:
# 差分計算用 DataFrame
calc_L3 = categorical.drop(["L3_md5", "Response"], axis=1)
diff_L3 = pd.DataFrame(index=calc_L3.index, columns=[])

In [220]:
calc_L3.fillna(0.0, inplace=True)
for famous_id in famous_ids:
    df = calc_L3 - calc_L3.loc[famous_id]
    df.where(df == 0.0, 1.0, inplace=True)
    key = "L3Cdiff_" + str(int(famous_id))
    diff_L3[key] = df.T.sum()
    print(famous_id)

16.0
304.0
382.0
137.0
162.0
581.0
7.0
4.0
257.0
52.0
103.0
766.0
599.0
28.0
400.0
1691.0
56.0
104.0
538.0
96.0


In [221]:
saigonotatakai = pd.DataFrame(response)
saigonotatakai["diff_min"] = diff_L3.T.min()
min1 = pd.DataFrame(saigonotatakai[saigonotatakai["Response"] == 1.0][
    "diff_min"].value_counts()).rename(columns={
        "diff_min": "min1"
    })
min0 = pd.DataFrame(saigonotatakai[saigonotatakai["Response"] == 0.0][
    "diff_min"].value_counts()).rename(columns={
        "diff_min": "min0"
    })
min10 = pd.concat([min1, min0], axis=1)
min10["ratio"] = min10["min1"] / min10.T.sum()
min10

,min1,min0,ratio
0.0,4867.0,995301,0.004866
1.0,139.0,23465,0.005889
2.0,1139.0,48691,0.022858
3.0,309.0,54244,0.005664
4.0,106.0,15835,0.006650
5.0,120.0,9169,0.012919
6.0,32.0,1571,0.019963
7.0,25.0,2733,0.009065
8.0,89.0,16663,0.005313
9.0,23.0,8032,0.002855
